# LGB超参数

In [ ]:
params = {'num_leaves': 31, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
          'objective': 'binary', #定义的目标函数
          'max_depth': -1,
          'learning_rate': 0.1,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 1,  #提取的特征比率
          "bagging_freq": 1,
          "bagging_fraction": 1,
          "bagging_seed": 11,
#           "lambda_l1": 0.1,             #l1正则
          # 'lambda_l2': 0.001,     #l2正则
          "verbosity": -1,
          "nthread": multiprocessing.cpu_count(),                #线程数量，-1表示全部线程，线程越多，运行的速度越快
          'metric': {'binary_logloss', 'auc'},  ##评价函数选择
          "random_state": 2077, #随机数种子，可以防止每次运行的结果不一致
          # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
          }

n_fold = 5
oof_pre = np.zeros(len(train_df))
y_pre = np.zeros(len(test_df))

kf = KFold(n_splits=n_fold)
for fold_, (trn_idx, val_idx) in enumerate(kf.split(train_df)):

    trn_data = lgb.Dataset(train_df[feature].iloc[trn_idx], label=train_df['label'].iloc[trn_idx])
    val_data = lgb.Dataset(train_df[feature].iloc[val_idx], label=train_df['label'].iloc[val_idx])

    clf = lgb.train(params,
                    trn_data,
                    2000,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=50,
                    early_stopping_rounds=50)
    # out of fold
    oof_pre[val_idx] = clf.predict(train_df[feature].iloc[val_idx], num_iteration=clf.best_iteration)

    y_pre += clf.predict(test_df[feature], num_iteration=clf.best_iteration) / n_fold

这段代码是使用LightGBM进行二分类任务的一个例子。它展示了如何设置参数、进行交叉验证、训练模型并进行预测。下面是详细的步骤和参数解释：

1. **参数设置**：
   - `params` 字典包含了LightGBM模型的参数。这些参数包括树的数量、学习率、正则化项等。
   - `'num_leaves'`：树的最大叶节点数，与模型的复杂度相关，太大可能导致过拟合。
   - `'objective'`：定义了优化的目标函数，这里是 `'binary'`，因为是二分类任务。
   - `'max_depth'`：树的最大深度，设置为 `-1` 表示不限制树的深度。
   - `'learning_rate'`：学习率，较小的值通常需要更多的树来学习相同的复杂度模型。
   - `'min_sum_hessian_in_leaf'`：叶节点最小的海森矩阵和（二阶导数的和），用于控制树的生长。
   - `'boosting'`：设置提升类型，这里是 `'gbdt'`，即传统的梯度提升决策树。
   - `'feature_fraction'`、`'bagging_freq'`、`'bagging_fraction'`、`'bagging_seed'`：这些参数与特征和数据子采样有关。
   - `'verbosity'`：控制训练过程中的输出信息，`-1` 表示不输出信息。
   - `'nthread'`：设置并行线程数，这里使用 `multiprocessing.cpu_count()` 自动检测并使用所有可用的CPU核心。
   - `'metric'`：评价函数，这里使用了二分类的对数损失和AUC。
   - `'random_state'`：随机数种子，确保结果可重复。

2. **交叉验证**：
   - 设置交叉验证的折数 `n_fold = 5`，这里使用了 `KFold` 而不是 `StratifiedKFold`，因此不保证每个折中类别的比例与整个数据集相同。
   - 初始化两个零数组 `oof_pre` 和 `y_pre`，分别用于存储训练集的预测结果（out-of-fold predictions）和测试集的预测结果。

3. **模型训练与预测**：
   - 在 `KFold` 的每一折中，分别提取训练集和验证集的索引 `trn_idx` 和 `val_idx`。
   - 使用这些索引来创建LightGBM的Dataset对象 `trn_data` 和 `val_data`。
   - 调用 `lgb.train` 来训练模型，设置迭代次数、训练集、验证集、早停条件等。
   - 使用最优迭代次数 `clf.best_iteration` 对验证集进行预测，并将结果存储在 `oof_pre[val_idx]` 中。
   - 使用最优迭代次数对测试集进行预测，并将结果累加到 `y_pre` 中，最终取平均得到最终的测试集预测。

4. **内存优化**：
   - 最后，调用之前定义的 `reduce_mem_usage` 函数对训练集和测试集进行内存优化。

通过这种方式，代码实现了交叉验证的同时避免了数据泄露，最终得到了对训练集和测试集的预测结果。这种方法在机器学习竞赛和实际项目中非常常见。